In [1]:
# CARGA DE DATOS 
# Cargamos los CSV desde la carpeta /data/raw.

import pandas as pd

df_pm10 = pd.read_csv('../data/raw/pm 10 coyhaique (extended).csv', sep=';', encoding='latin1', engine='python')
df_pm25 = pd.read_csv('../data/raw/pm 2.5 coyhaique.csv', sep=';')
df_co = pd.read_csv('../data/raw/CO coyhaique.csv', sep=';')
df_so2 = pd.read_csv('../data/raw/SO2 coyahique.csv', sep=';')
df_meteo = pd.read_csv('../data/raw/export (extended).csv')
df_umbrales = pd.read_csv('../data/raw/umbrales_pm10.csv', sep=';')

print("Archivos cargados correctamente.")

Archivos cargados correctamente.


# Importamos Funciones de preprocesamiento 

In [2]:
# Permitir importar desde la carpeta src/
import sys
import os

src_path = os.path.abspath(os.path.join('..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

# importar
from data.processing import procesar_contaminante, procesar_meteorologia

In [3]:
# 🔹 IMPORTAMOS también las funciones nuevas
from data.processing import procesar_contaminante, procesar_meteorologia, unir_datasets

# 🔹 PROCESAMOS Y UNIMOS LOS DATOS
df_total = unir_datasets(df_pm10, df_pm25, df_co, df_so2, df_meteo, df_umbrales)

# 🔎 VERIFICAMOS LA SALIDA
print("✅ Dataset final unificado con forma:", df_total.shape)
print("Columnas disponibles:", df_total.columns.tolist())
print("Distribución de clases de riesgo:")
print(df_total['Riesgo'].value_counts())


KeyError: 'PM10'

from data.processing import procesar_contaminante, procesar_meteorologia, unir_datasets

# 🔹 PROCESAMOS Y UNIMOS LOS DATOS
df_total = unir_datasets(df_pm10, df_pm25, df_co, df_so2, df_meteo, df_umbrales)

# 🔎 VERIFICAMOS LA SALIDA
print("✅ Dataset final unificado con forma:", df_total.shape)
print("Columnas disponibles:", df_total.columns.tolist())
print("Distribución de clases de riesgo:")
print(df_total['Riesgo'].value_counts())